<a href="https://colab.research.google.com/github/pardeshigithub/GIS-System/blob/main/Manipulating_Geospatial_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'geospatial-learn-course-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F348259%2F695175%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T143902Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0cb4bd9446dabb61e7abcdf55c102963ef5d7710ef1a13c7ebdca8df6a81d32a8beac2bd52668695ac4406bdfc20bb2d45b4a5693a4720a50536529f7aae6a1313cff361962145f2097a5c615bea6acc66cdbef1733702ff842997687c2e2f8072c0e4a79ccea55f7277cfc12445fbffd640a922afda08bc1aa7de782e24e2eae5b96d04ac71e621ef280bb715f35732caca7e728071301391d0b5f07743bdaf33ff587ec109f80e1877b4fc3bab10561112ed59e966261fd1e386984dd5b9340dfd5b08ceb5764fd0bb26cf1d498b61932e05bb84764b697e2c943448e45146ed28960b3fc55fa6e3e9edfc886e874abaa39433413a37bf7a609071e97872be,data-for-datavis:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F116573%2F3551030%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T143902Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D89bf65c8bf6ceea7456ea66ecde0c70628c6e7624eaa12c41f4c7d1de09c59bfc4f1a0d4d2d5bab50a281639a5420e9ef3bf1ffbd54640923e12bbcac1dcefd75e02a2bc808b8541f455e7c7558bc7ca4ae762074032beec2e392b3872a59b32157c8b86fd661d2c46c75102321909c0f3b78285997ada57585ce25de0a733e2ceddf8b6bef9a6d457814d5450e57397283ff63210668f5c3473c434bb89417276a459b02207dd8c7d683502f19315697e450e5729a968a82ae1c9b11b0b11fac2636fdb010455992c2b1550cc169982b3451091196a7a287052479bbea1999a64dac67a6bbf0b293414dc57bed377b15c09832eb90f81517299c91235715406'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 244538909 bytes downloaded
Downloaded and uncompressed: geospatial-learn-course-data
[==================================================] 145429 bytes downloaded
Downloaded and uncompressed: data-for-datavis
Data source import complete.


# Introduction

In this tutorial, you'll learn about two common manipulations for geospatial data: **geocoding** and **table joins**.

In [2]:

import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings
warnings.filterwarnings('ignore')

# Geocoding

**Geocoding** is the process of converting the name of a place or an address to a location on a map.  If you have ever looked up a geographic location based on a landmark description with [Google Maps](https://www.google.com/maps), [Bing Maps](https://www.bing.com/maps), or [Baidu Maps](https://map.baidu.com/), for instance, then you have used a geocoder!

![](https://storage.googleapis.com/kaggle-media/learn/images/1IrgZQq.png)

We'll use geopy to do all of our geocoding.

In [8]:
from geopy.geocoders import Nominatim

In the code cell above, [`Nominatim`](https://nominatim.openstreetmap.org/) refers to the geocoding software that will be used to generate locations.

We begin by instantiating the geocoder.  Then, we need only apply the name or address as a Python string. (In this case, we supply `"Pyramid of Khufu"`, also known as the Great Pyramid of Giza.)

If the geocoding is successful, it returns a `geopy.location.Location` object with two important attributes:
- the "point" attribute contains the (latitude, longitude) location, and
- the "address" attribute contains the full address.

In [9]:
geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Aarya Residency Pune")

print(location.point)
print(location.address)

18 28m 11.4773s N, 73 48m 38.9326s E
Aarya Residency, Prayeja City Road, Manik Baug, Pune City, Pune, Maharashtra, 411041, India


The value for the "point" attribute is a `geopy.point.Point` object, and we can get the latitude and longitude from the `latitude` and `longitude` attributes, respectively.

In [ ]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

It's often the case that we'll need to geocode many different addresses.  For instance, say we want to obtain the locations of 100 top universities in Europe.

In [10]:
universities = pd.read_csv("../input/geospatial-learn-course-data/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


Then we can use a lambda function to apply the geocoder to every row in the DataFrame.  (We use a try/except statement to account for the case that the geocoding is unsuccessful.)

In [11]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = {'init': 'epsg:4326'}
universities.head()

90.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.210946,0.092005,POINT (0.09200 52.21095)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.413218,8.537491,POINT (8.53749 47.41322)
4,UCL,51.521785,-0.135151,POINT (-0.13515 51.52179)


Next, we visualize all of the locations that were returned by the geocoder.  Notice that a few of the locations are certainly inaccurate, as they're not in Europe!

In [ ]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

# Table joins

Now, we'll switch topics and think about how to combine data from different sources.  

### Attribute join

[You already know](https://www.kaggle.com/residentmario/renaming-and-combining) how to use `pd.DataFrame.join()` to combine information from multiple DataFrames with a shared index.  We refer to this way of joining data (by simpling matching values in the index) as an **attribute join**.

When performing an attribute join with a GeoDataFrame, it's best to use the `gpd.GeoDataFrame.merge()`.  To illustrate this, we'll work with a GeoDataFrame `europe_boundaries` containing the boundaries for every country in Europe.  The first five rows of this GeoDataFrame are printed below.

In [ ]:

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [ ]:
europe_boundaries.head()

We'll join it with a DataFrame `europe_stats` containing the estimated population and gross domestic product (GDP) for each country.

In [ ]:
europe_stats.head()

We do the attribute join in the code cell below.  The `on` argument is set to the column name that is used to match rows in `europe_boundaries` to rows in `europe_stats`.

In [ ]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

### Spatial join

Another type of join is a **spatial join**.  With a spatial join, we combine GeoDataFrames based on the spatial relationship between the objects in the "geometry" columns.  For instance, we already have a GeoDataFrame `universities` containing geocoded addresses of European universities.  

Then we can use a spatial join to match each university to its corresponding country.  We do this with `gpd.sjoin()`.

In [ ]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

The spatial join above looks at the "geometry" columns in both GeoDataFrames.  If a Point object from the `universities` GeoDataFrame intersects a Polygon object from the `europe` DataFrame, the corresponding rows are combined and added as a single row of the `european_universities` DataFrame.  Otherwise, countries without a matching university (and universities without a matching country) are omitted from the results.

The `gpd.sjoin()` method is customizable for different types of joins, through the `how` and `op` arguments.  For instance, you can do the equivalent of a SQL left (or right) join by setting `how='left'` (or `how='right'`).  We won't go into the details in this course, but you can learn more in [the documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html).

# Your turn

**[Use geocoding and table joins](https://www.kaggle.com/kernels/fork/5832170)** to identify suitable locations for the next Starbucks Reserve Roastery.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*